In [14]:
import numpy as np
import random as rd
import pandas as pd
from time import perf_counter
from scipy.special import comb

In [15]:
MAX_FITNESS = 28
MAX_GENERATION = 1000
POP_SIZE = 20
CROSSOVER_PROB = 80
MUTATION_PROB = 3
ITERATIONS_RANGE = 50

In [16]:
def GA_fitness_function(board: list) -> int: 
    """
    Retorna o número representante de fitness de um indivíduo candidato.
    O valor fitness representa o número de pares não atacantes de uma resposta:
    (0 - todas as rainhas se atacam, 28 - nenhum par de rainhas se ataca)
    """
    n = len(board)
    c = MAX_FITNESS
    
    board = [int(i,2) for i in board]
    

    row_frequency = [0] * n
    main_diag_frequency = [0] * (2*n)
    sec_diag_frequency = [0] * (2*n)

    for i in range(n):
        row_frequency[board[i]] += 1
        main_diag_frequency[board[i] + i] += 1
        sec_diag_frequency[n - board[i] + i] += 1

    attacks = 0

    for i in range(2*n):
        if i < n:
            attacks += (row_frequency[i] * (row_frequency[i]-1)) 
        
        attacks += (main_diag_frequency[i] * (main_diag_frequency[i]-1)) 
        attacks += (sec_diag_frequency[i]  * (sec_diag_frequency[i]-1))

    
    return int(c - attacks/2)


def GA_Parent_Selection(pop: list) -> list:
    """
    Retorna dois indivíduos de uma população, comparando com o
    fitness de cada um. A seleção ocorre por meio do tipo Rodeio, no
    qual a probabilidade de escolha é proporcional ao fitness do
    indivíduo. (roleta)
    """
    probabilities = []
    # calcula o fitness total da população
    overall_fitness = sum(individual[1] for individual in pop)
    
    for individual in pop:
        # salva a probabilidade deste indivíduo ser escolhido por meio de seu fitness
        individualProb = individual[1] / overall_fitness
        probabilities.append(individualProb)
    
    # escolhe e retorna dois indivíduos parentes
    parentList = rd.choices(pop, weights=probabilities, k=2)
    return parentList


def GA_Crossover(parent_1, parent_2, pop) -> tuple:
    """
    Retorna dois novos filhos à população por meio do crossover do tipo ponto de corte
    entre dois pais. O ponto de corte é escolhido de forma aleatória.
    """
    # Gera um ponto de corte aleatório e repassa os bits gerados no corte
    cutPoint = rd.randint(1,7)
    x1 = parent_1[0][:cutPoint] + parent_2[0][cutPoint:]
    x2 = parent_2[0][:cutPoint] + parent_1[0][cutPoint:]

    # Gera dois descendentes com os bits gerados no corte, mais o seu fitness
    individual1 = [x1, GA_fitness_function(x1)]
    individual2 = [x2, GA_fitness_function(x2)]

    return individual1, individual2


def GA_bit_flip(individual):
    """
    Método de mutação, muda (de 0 para 1 ou 1 para 0) um valor aleatório 
    de um indivíduo, recebe um indivíduo e retorna o indivíduo mutado.
    """
    # escolhe aleatoriamente um ponto de mutação
    num = rd.randint(0, 2)
    value = rd.choice(individual[0])
    column = individual[0].index(value)
    
    # Realiza a mutação
    flip = "0" if value[num] == "1" else "1"

    # retorna a mutação ao indivíduo mutado, mais o seu novo fitness
    individual[0][column] = (individual[0][column][:num] 
                             + flip 
                             + individual[0][column][num+1:])
    
    individual[1] = GA_fitness_function(individual[0])
    

def GA_Survivors_Selection(pop, new_pop):
    """
    Retorna os indivíduos com os 20 maiores fitness entre os grupos 
    da população original e a população descentente (elitista)
    """ 
    # junta e ordena a população de descendentes 
    # e a atual por fitness decrescentemente

    total = pop + new_pop
    total.sort(key=lambda x:x[1], reverse=True)

    # retorna a população da nova geração
    return total[:POP_SIZE]


def GA_Pop_Creator():
    """
    Inicia uma população de 20 indivíduos, cada indivíduo apresenta 8 grupos (de 0 a 7)
    de bits 0s e 1s em formato de string, representando as posições verticais no tabuleiro.
    ex. de indivíduo gerado: [['000', '011', '110', '100', '101', '010', '001', '111'], 0]
    """
    pop = []

    # Inicializa população com 20 indivíduos
    for i in range(0, POP_SIZE):          

        # gera uma resposta aleatória a cada indivíduo, sem repetir o número das linhas (heurística)                           
        board = rd.sample(range(8), 8)
        board = [f"{bin(x)}"[2:].zfill(3) for x in board]

        pop.append([board, GA_fitness_function(board)])

    return pop


def genetic_algorithm():
    """
    Função principal retorna uma resposta ótima dentro de 1000 gerações
    caso as gerações passem de 1000 sem uma resposta, retorna a melhor resposta presente
    """

    generation = 0
    new_pop = []

    # Inicialização da população com 20 indivíduos aleatórios
    pop = GA_Pop_Creator()         
    
    
    while generation < MAX_GENERATION:
        
        # retorna uma resposta ótima caso tenha encontrado
        for individual in pop:
            if individual[1] == MAX_FITNESS:
                return individual, generation
        
        # gera o crossover na população atual
        for _ in range(0, POP_SIZE // 2):
            parent1, parent2 = GA_Parent_Selection(pop)

            random_num = rd.randint(1,100)
            if random_num <= CROSSOVER_PROB:

                # indivíduos criados no crossover entram na população de descendentes 
                individual1, individual2 = GA_Crossover(parent1,parent2,pop)    
                new_pop.extend([individual1, individual2])
                
            else:

                # Pais entram na população descendente
                new_pop.extend([parent1, parent2])
                
        # gera a mutação na população de descendentes
        for i in new_pop:
            random_num = rd.randint(1, 100)

            if random_num <= MUTATION_PROB:
                GA_bit_flip(i)

        # gera a população da nova geração
        pop = GA_Survivors_Selection(pop, new_pop)
        generation += 1

    #Indivíduo com o melhor fitness da população
    best_individual = max(pop, key=lambda x: x[1])
    
    return best_individual, generation


#salvamento dos dados em uma lista/tabela como resposta final:

if __name__ == "__main__":
    
    finalTable = []
    for i in range(ITERATIONS_RANGE):

        t_start = perf_counter()
        answer, num_generation = genetic_algorithm()
        t_final = perf_counter()

        exec_time = (t_final - t_start) * 1000 #Conversão para milissegundos

        result = [i+1,
                  answer[0],
                  answer[1],
                  round(exec_time, 2),
                  num_generation
            ]
        
        finalTable.append(result)


In [17]:
def generate_statistics_dataframes(return_dfs: bool,
                                   finalTable = finalTable):
    
    st_indexes = ["Iteração",
                  "Melhor solução",
                  "Fitness (Melhor solução)",
                  "Tempo de execução (ms)",
                  "Nº da geração em que o algoritmo parou"
        ]

    GA_data = pd.DataFrame(finalTable, columns=st_indexes)

    st_data = []
    for column in st_indexes[2:]:
        st_data.append([GA_data[column].mean(), GA_data[column].std()])


    indexes = ["Média e desvio padrão dos melhores fitnesses",
            "Média e desvio padrão dos tempos de execução",
            "Média e desvio padrão dos números de geração"]

    statistics = pd.DataFrame(st_data, index=indexes, 
                            columns=["Média", "Desvio padrão"])

    display(GA_data)
    print()
    display(statistics.round(2))

    if return_dfs:
        return GA_data, statistics.round(2)

generate_statistics_dataframes(False)

,Iteração,Melhor solução,Fitness (Melhor solução),Tempo de execução (ms),Nº da geração em que o algoritmo parou
0,1,"[101, 000, 100, 001, 111, 010, 110, 011]",28,5400.47,510
1,2,"[000, 100, 111, 101, 010, 110, 001, 011]",28,0.64,0
2,3,"[011, 101, 111, 010, 100, 110, 001, 101]",27,20321.50,1000
3,4,"[100, 110, 011, 000, 010, 111, 101, 001]",28,4179.08,452
4,5,"[100, 001, 101, 000, 110, 011, 111, 010]",28,1326.31,250
5,6,"[011, 110, 000, 111, 100, 001, 101, 010]",28,195.08,91
6,7,"[100, 000, 111, 101, 010, 110, 001, 011]",28,104.69,61
7,8,"[100, 010, 000, 110, 001, 111, 101, 011]",28,0.55,0
8,9,"[100, 001, 111, 000, 011, 110, 010, 101]",28,4698.57,476
9,10,"[011, 001, 110, 100, 000, 111, 101, 010]",28,3771.02,426


,Média,Desvio padrão
Média e desvio padrão dos melhores fitnesses,27.98,0.14
Média e desvio padrão dos tempos de execução,2359.48,3373.80
Média e desvio padrão dos números de geração,269.54,206.67
